In [64]:
import pandas as pd
import numpy as np

deliveries = pd.read_csv('../data/raw/deliveries.csv')
matches = pd.read_csv('../data/raw/matches.csv')


In [65]:
deliveries.shape 
matches.shape

(636, 18)

In [66]:
print (deliveries.head())
print (matches.head)


   match_id  inning         batting_team                 bowling_team  over  \
0         1       1  Sunrisers Hyderabad  Royal Challengers Bangalore     1   
1         1       1  Sunrisers Hyderabad  Royal Challengers Bangalore     1   
2         1       1  Sunrisers Hyderabad  Royal Challengers Bangalore     1   
3         1       1  Sunrisers Hyderabad  Royal Challengers Bangalore     1   
4         1       1  Sunrisers Hyderabad  Royal Challengers Bangalore     1   

   ball    batsman non_striker    bowler  is_super_over  ...  bye_runs  \
0     1  DA Warner    S Dhawan  TS Mills              0  ...         0   
1     2  DA Warner    S Dhawan  TS Mills              0  ...         0   
2     3  DA Warner    S Dhawan  TS Mills              0  ...         0   
3     4  DA Warner    S Dhawan  TS Mills              0  ...         0   
4     5  DA Warner    S Dhawan  TS Mills              0  ...         0   

   legbye_runs  noball_runs  penalty_runs  batsman_runs  extra_runs  \
0        

In [67]:
matches.columns


Index(['id', 'season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2',
       'umpire3'],
      dtype='object')

In [68]:
deliveries.columns


Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')

In [69]:
# data merge 
merged_df = deliveries.merge(
    matches[['id', 'season']],
    left_on='match_id',
    right_on='id',
    how='left'
)


In [70]:
merged_df.shape
merged_df.head()


,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder,id,season
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,NaN,NaN,NaN,1,2017
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,NaN,NaN,NaN,1,2017
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,4,0,4,NaN,NaN,NaN,1,2017
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,NaN,NaN,NaN,1,2017
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,2,2,NaN,NaN,NaN,1,2017


In [71]:
# top batman in last 10 years (# batting roles)
# opener
# middle order 
# lower  order 

In [72]:
# total runs
batsman_runs = (
    merged_df
    .groupby('batsman')['batsman_runs']
    .sum()
    .sort_values(ascending=False)
)


In [73]:
# balls faced 
balls_faced = (
    merged_df[merged_df['wide_runs'] == 0]
    .groupby('batsman')
    .size()
)


In [74]:
# strike rate
strike_rate = (batsman_runs / balls_faced) * 100


In [75]:
# batsman summery 
batsman_summary = pd.DataFrame({
    'runs': batsman_runs,
    'balls': balls_faced,
    'strike_rate': strike_rate
}).dropna()


In [76]:
batsman_summary.sort_values('runs', ascending=False).head(10)


,runs,balls,strike_rate
batsman,,,
SK Raina,4548,3267,139.210285
V Kohli,4423,3405,129.897210
RG Sharma,4207,3214,130.896080
G Gambhir,4132,3316,124.607961
DA Warner,4014,2825,142.088496
RV Uthappa,3778,2870,131.637631
CH Gayle,3651,2409,151.556663
S Dhawan,3561,2922,121.868583
MS Dhoni,3560,2604,136.712750


In [77]:

legal_df = merged_df[merged_df['wide_runs'] == 0]


In [78]:
batsman_runs = legal_df.groupby('batsman')['batsman_runs'].sum()
balls_faced = legal_df.groupby('batsman').size()

batsman_df = pd.DataFrame({
    'runs': batsman_runs,
    'balls': balls_faced
})

batsman_df['strike_rate'] = (batsman_df['runs'] / batsman_df['balls']) * 100


In [79]:
batsman_df.sort_values('runs', ascending=False).head()


,runs,balls,strike_rate
batsman,,,
SK Raina,4548,3267,139.210285
V Kohli,4423,3405,129.897210
RG Sharma,4207,3214,130.896080
G Gambhir,4132,3316,124.607961
DA Warner,4014,2825,142.088496


In [80]:
# for top order 
top_order = batsman_df[
    (batsman_df['runs'] >= 4000) &
    (batsman_df['strike_rate'] >= 125)
].sort_values(['runs', 'strike_rate'], ascending=False)


In [81]:
top_order


,runs,balls,strike_rate
batsman,,,
SK Raina,4548,3267,139.210285
V Kohli,4423,3405,129.897210
RG Sharma,4207,3214,130.896080
DA Warner,4014,2825,142.088496


In [82]:
middle_order = batsman_df[
    (batsman_df['runs'] >= 3000) &
    (batsman_df['strike_rate'] >= 135)
].sort_values(['runs', 'strike_rate'], ascending=False)


In [83]:
middle_order


,runs,balls,strike_rate
batsman,,,
SK Raina,4548,3267,139.210285
DA Warner,4014,2825,142.088496
CH Gayle,3651,2409,151.556663
MS Dhoni,3560,2604,136.712750
AB de Villiers,3486,2348,148.466780


In [84]:
batsman_df['role'] = 'Other'

batsman_df.loc[top_order.index, 'role'] = 'Top Order'
batsman_df.loc[middle_order.index, 'role'] = 'Middle Order'


In [85]:
batsman_df[batsman_df['role'] != 'Other']


,runs,balls,strike_rate,role
batsman,,,,
AB de Villiers,3486,2348,148.466780,Middle Order
CH Gayle,3651,2409,151.556663,Middle Order
DA Warner,4014,2825,142.088496,Middle Order
MS Dhoni,3560,2604,136.712750,Middle Order
RG Sharma,4207,3214,130.896080,Top Order
SK Raina,4548,3267,139.210285,Middle Order
V Kohli,4423,3405,129.897210,Top Order


In [86]:
best_batsmen = batsman_df[batsman_df['role'] != 'Other']

best_batsmen.reset_index().to_csv(
    '../data/processed/best_xi_batsmen_metrics.csv',
    index=False
)


In [87]:
legal_deliveries = deliveries[
    (deliveries['wide_runs'] == 0) &
    (deliveries['noball_runs'] == 0)
]


In [88]:
# batting matrics for an all rounder 
batting = (
    legal_deliveries
    .groupby('batsman')
    .agg(
        runs=('batsman_runs', 'sum'),
        balls=('batsman_runs', 'count')
    )
    .reset_index()
)

batting['strike_rate'] = (batting['runs'] / batting['balls']) * 100


In [89]:
batting.sort_values('runs', ascending=False).head(10)


,batsman,runs,balls,strike_rate
374,SK Raina,4526,3251,139.218702
431,V Kohli,4412,3394,129.994107
323,RG Sharma,4184,3203,130.627537
137,G Gambhir,4112,3306,124.379915
103,DA Warner,3985,2809,141.865433
340,RV Uthappa,3767,2855,131.943958
85,CH Gayle,3610,2394,150.793651
259,MS Dhoni,3553,2593,137.022754
347,S Dhawan,3541,2913,121.558531
22,AB de Villiers,3460,2340,147.863248


In [90]:
deliveries['runs_conceded'] = (
    deliveries['total_runs']
    - deliveries['bye_runs']
    - deliveries['legbye_runs']
)


In [91]:
wickets = deliveries[
    (deliveries['player_dismissed'].notna()) &
    (deliveries['dismissal_kind'] != 'run out')
]


In [92]:
legal_deliveries = legal_deliveries.copy()

legal_deliveries.loc[:, 'runs_conceded'] = (
    legal_deliveries['total_runs']
    - legal_deliveries['bye_runs']
    - legal_deliveries['legbye_runs']
)


In [93]:
legal_deliveries[['total_runs','bye_runs','legbye_runs','runs_conceded']].head()


,total_runs,bye_runs,legbye_runs,runs_conceded
0,0,0,0,0
1,0,0,0,0
2,4,0,0,4
3,0,0,0,0
5,0,0,0,0


In [94]:
# bowler version
bowling = (
    legal_deliveries
    .groupby('bowler')
    .agg(
        balls=('ball', 'count'),
        runs_conceded=('runs_conceded', 'sum')
    )
    .reset_index()
)


In [95]:
wicket_count = (
    legal_deliveries[legal_deliveries['player_dismissed'].notna()]
    .groupby('bowler')
    .size()
    .reset_index(name='wickets')
)


In [96]:
bowling = bowling.merge(wicket_count, on='bowler', how='left')
bowling['wickets'] = bowling['wickets'].fillna(0)


In [97]:
bowling['overs'] = bowling['balls'] / 6
bowling['economy'] = bowling['runs_conceded'] / bowling['overs']

bowling['bowling_strike_rate'] = bowling['balls'] / bowling['wickets']
bowling.loc[bowling['wickets'] == 0, 'bowling_strike_rate'] = None


In [98]:
bowling.sort_values('wickets', ascending=False).head()


,bowler,balls,runs_conceded,wickets,overs,economy,bowling_strike_rate
294,SL Malinga,2558,2748,168.0,426.333333,6.445661,15.226190
5,A Mishra,2639,3170,142.0,439.833333,7.207275,18.584507
88,DJ Bravo,2018,2648,136.0,336.333333,7.873142,14.838235
229,PP Chawla,2563,3198,133.0,427.166667,7.486539,19.270677
117,Harbhajan Singh,2919,3292,132.0,486.500000,6.766701,22.113636


In [99]:
# top bowlers 
qualified_bowlers = bowling[
    (bowling['wickets'] >= 50) &
    (bowling['balls'] >= 300)
]


In [100]:
qualified_bowlers['rank_wickets'] = qualified_bowlers['wickets'].rank(ascending=False)
qualified_bowlers['rank_economy'] = qualified_bowlers['economy'].rank()
qualified_bowlers['rank_sr'] = qualified_bowlers['bowling_strike_rate'].rank()

qualified_bowlers['overall_rank'] = (
    qualified_bowlers['rank_wickets'] +
    qualified_bowlers['rank_economy'] +
    qualified_bowlers['rank_sr']
)


C:\Users\Madhav\AppData\Local\Temp\ipykernel_9884\3716435980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified_bowlers['rank_wickets'] = qualified_bowlers['wickets'].rank(ascending=False)
C:\Users\Madhav\AppData\Local\Temp\ipykernel_9884\3716435980.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified_bowlers['rank_economy'] = qualified_bowlers['economy'].rank()
C:\Users\Madhav\AppData\Local\Temp\ipykernel_9884\3716435980.py:3: SettingWithCopyWarning: 
A value is trying to be set on a co

In [101]:
top_bowlers = qualified_bowlers.sort_values('overall_rank').head(5)
top_bowlers


,bowler,balls,runs_conceded,wickets,overs,economy,bowling_strike_rate,rank_wickets,rank_economy,rank_sr,overall_rank
294,SL Malinga,2558,2748,168.0,426.333333,6.445661,15.226190,1.0,5.0,3.0,9.0
50,B Kumar,1981,2238,117.0,330.166667,6.778395,16.931624,9.0,7.0,12.0,28.0
300,SP Narine,1931,1999,109.0,321.833333,6.211289,17.715596,10.0,1.0,17.0,28.0
7,A Nehra,1908,2405,120.0,318.000000,7.562893,15.900000,7.0,25.0,4.0,36.0
5,A Mishra,2639,3170,142.0,439.833333,7.207275,18.584507,2.0,10.0,25.0,37.0


In [104]:
top_bowlers.to_csv(
    'data/processed/best_xi_bowlers.csv',
    index=False
)

OSError: Cannot save file into a non-existent directory: 'data\processed'